## Exercise 4.5 Data Consistency
    Mixed data types, Missing data, duplicates, tidy up and export
    df included: df_prods, df_ords

In [5]:
#import libraries
import pandas as pd
import numpy as np
import os as os

### Create a test dataframe with mixed data types

In [3]:

df_test = pd.DataFrame()

In [4]:
# Create a mixed type column
df_test['mix']= ['a','b', 1, True]

In [5]:
df_test.head()

,mix
0,a
1,b
2,1
3,True


### Check for mixed types

In [6]:
#check for mixed types
for col in df_test.columns.tolist():
    weird = (df_test[[col]].applymap(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_test[weird]) > 0:
        print(col)

mix


### Change data types to all strings
    df['mix'] = df['mix].astype('str')

In [7]:
df_test['mix'] = df_test['mix'].astype('str')

### Import product file

In [8]:
path = r'/Users/margoshogren/Desktop/1 Margo-CF Folder/4 Python/4-2023 Instacart Basket Analysis'
df_prods = pd.read_csv(os.path.join(path,'2 Data','original data','products.csv'),index_col = False)

## Find msg values
    df.isnull().sum()    #Note: sum needed to get more then True/False result

In [9]:
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

### product_name has 16 missing values, view msg values
    df_nan = df[df['col'].isnull() == True]

In [10]:
df_nan = df_prods[df_prods['product_name'].isnull() == True]

In [11]:
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


### Check products shape   #Note: Always check shape before & after filtering missing values
     df.shape

In [12]:
df_prods.shape

(49693, 5)

### Filter out missing values
    df_clean = df[df['col w/msg'].isnull() == False]

In [13]:
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [14]:
df_prods_clean.shape  #check shape after filtering out missing

(49677, 5)

#### Observation: There are 16 less rows after filtering out missing

### Check df_prods for duplicate values
    df = df_clean[df_clean.duplicated()]

In [15]:
df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [16]:
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


#### Observation: There are 5 duplicate rows

### Check df_prods_clean shape, so we can compare after removing duplicates
        df.shape

In [17]:
df_prods_clean.shape

(49677, 5)

### Remove duplicates
    df_clean = df_clean.drop_duplicates()

In [18]:
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

### Check shape after removing dups
    df.shape

In [19]:
df_prods_clean_no_dups.shape

(49672, 5)

#### Observation: There are 5 less rows after removing the duplicates

### Check for other data anomalies or outliers on products file

In [20]:
df_prods_clean_no_dups.describe()

,product_id,aisle_id,department_id,prices
count,49672.000000,49672.000000,49672.000000,49672.000000
mean,24850.349775,67.762442,11.728942,9.993282
std,14340.705287,38.315784,5.850779,453.615536
min,1.000000,1.000000,1.000000,1.000000
25%,12432.750000,35.000000,7.000000,4.100000
50%,24850.500000,69.000000,13.000000,7.100000
75%,37268.250000,100.000000,17.000000,11.100000
max,49688.000000,134.000000,21.000000,99999.000000


#### Observation: the max prices at 99999. is much higher than the mean or average. We need to check the prices values.
#### Check for prices > $50

In [21]:
df_prods_clean_no_dups[df_prods_clean_no_dups["prices"]> 50.0]

,product_id,product_name,aisle_id,department_id,prices
21554,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0
33666,33664,2 % Reduced Fat Milk,84,16,99999.0


#### We will replace the 14900 (product_id = 21553) with 1.49  and the 99999.0 (product id = 33666) with 9.99 (since it looks like the decimal was omitted)

In [22]:
df_prods_clean_no_dups = df_prods_clean_no_dups.replace ({'prices':{99999.0: 9.99, 14900.0:1.49 }})

In [23]:
df_prods_clean_no_dups[df_prods_clean_no_dups["prices"]> 50.0]

,product_id,product_name,aisle_id,department_id,prices


In [24]:
df_prods_clean_no_dups.describe()

,product_id,aisle_id,department_id,prices
count,49672.000000,49672.000000,49672.000000,49672.000000
mean,24850.349775,67.762442,11.728942,7.680359
std,14340.705287,38.315784,5.850779,4.199401
min,1.000000,1.000000,1.000000,1.000000
25%,12432.750000,35.000000,7.000000,4.100000
50%,24850.500000,69.000000,13.000000,7.100000
75%,37268.250000,100.000000,17.000000,11.100000
max,49688.000000,134.000000,21.000000,25.000000


#### Observation:  Now the max is closer to the mean

### Import orders.csv file

In [6]:
path = r'/Users/margoshogren/Desktop/1 Margo-CF Folder/4 Python/4-2023 Instacart Basket Analysis'
df_ords = pd.read_csv(os.path.join(path,'2 Data','original data','orders.csv'),index_col = False)

### Question(2) Describe orders.csv
    df.describe()

In [26]:
df_ords.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


### Observation:  0 min value of order_dow (day of week) is 24 hr time and is midnight.  

### Question(3) Check for mixed-type data in your df_ords dataframe.

In [27]:
for col in df_ords.columns.tolist():
    weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_ords[weird]) > 0:
        print(col)

### Question(4) no mixed data types were found

### Question(5) Run a check for missing values in your df_ords dataframe 
    df.isnull().sum()

In [3]:
df_ords.isnull().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

### Question(5- continued) View the msg data, for order 1
    df_nan = df[df['col'].isnull() == True],[df['element'] == 'element']

In [29]:
df_ords_nan = df_ords[df_ords['days_since_prior_order'].isnull() == True],[df_ords['order_number'] == 1]

In [30]:
df_ords_nan

(         order_id  user_id eval_set  order_number  order_dow  \
 0         2539329        1    prior             1          2   
 11        2168274        2    prior             1          2   
 26        1374495        3    prior             1          1   
 39        3343014        4    prior             1          6   
 45        2717275        5    prior             1          3   
 ...           ...      ...      ...           ...        ...   
 3420930    969311   206205    prior             1          4   
 3420934   3189322   206206    prior             1          3   
 3421002   2166133   206207    prior             1          6   
 3421019   2227043   206208    prior             1          1   
 3421069   3154581   206209    prior             1          3   
 
          order_hour_of_day  days_since_prior_order  
 0                        8                     NaN  
 11                      11                     NaN  
 26                      14                     NaN  
 3

#### Observation: all the missing values  for 'days since prior order' are for order 1, these were probably omitted when starting data input and can be left missing

In [31]:
df_ords.shape

(3421083, 7)

In [32]:

df_ords.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


#### Observation: The ranges of these values do not show any anomalies

### Question(7) Run a check for duplicate values in your df_ords data.
    df_dups = df_clean[df_clean.duplicated()]

In [7]:
df_ords_dups = df_ords[df_ords.duplicated()]

In [34]:
df_ords_dups

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order


In [9]:
df_ords.shape

(3421083, 7)

#### Observation: There are no duplicate rows in the order file.

### Question(8) Address the duplicates using an appropriate method

In [37]:
df_prods_clean_no_dups.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


### Question(9) Export your final, cleaned df_prods and df_ords data as “.csv” files in your “Prepared Data” folder and give them appropriate, succinct names.
    df.to_csv(os.path.join(path,'2 Data','prepared data','products_checked.csv'))

In [35]:
df_prods_clean_no_dups.to_csv(os.path.join(path,'2 Data','prepared data','products_checked.csv'))

In [36]:
df_ords_dups.to_csv(os.path.join(path,'2 Data','prepared data','orders_checked.csv'))

### --End of Notebook